In [1]:
import os
import openai

import pandas as pd
import numpy as np

from rouge_score import rouge_scorer

openai.api_key = os.environ["OPENAI_API_KEY"]

/Users/johngalvin/miniforge3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load Data

In [2]:
df = pd.read_csv("../data/processed/processed.csv")

In [3]:
df

,dialogue,ground_truth
0,Amanda: I baked cookies. Do you want some? ...,Amanda baked cookies and will bring Jerry some...
1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,"Tim: Hi, what's up? Kim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,Sam: hey overheard rick say something Sam:...,"Sam is confused, because he overheard Rick com..."
...,...,...
14727,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,Theresa: <file_photo> Theresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,Jennifer: Dear Celia! How are you doing? Je...,Celia couldn't make it to the afternoon with t...


## Zero Shot Learning

In [25]:
zero_shot_summs = []

for i in range(len(df["dialogue"])):
    content = "Summarize this: " + df.iloc[i, 0]
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=content,
        max_tokens=32,
        temperature=0)
    zero_shot_summs.append(response.choices[0].text)

df["zero_shot_summaries"] = zero_shot_summs

Amanda: I baked  cookies. Do you want some?    Jerry: Sure!    Amanda: I'll bring you tomorrow :-)
Olivia: Who are you voting for in this election?     Oliver: Liberals as always.    Olivia: Me too!!    Oliver: Great
Tim: Hi, what's up?    Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating    Tim: What did you plan on doing?    Kim: Oh you know, uni stuff and unfucking my room    Kim: Maybe tomorrow I'll move my ass and do everything    Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies    Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores    Tim: It really helps    Kim: thanks, maybe I'll do that    Tim: I also like using post-its in kaban style
Edward: Rachel, I think I'm in ove with Bella..    rachel: Dont say anything else..    Edward: What do you mean??    rachel: Open your fu**ing door.. I'm outside
Sam: hey  overheard rick say something    Sam: i don't know what to do :-/

Samantha: hey, have you seen this    Samantha: <file_other>    Benjamin: oh it got cheaper, we need to buy it now!    Samantha: <file_gif>
Jill: are you there yet, June?    June: almost, 5 mins    Jill: I'll be a lil late. Can you get a nice table for us?    June: ok, no prob
Tori: I have just downloaded the pics    Omari: Can you send them to me?    Tori: wait    Omari: k    Tori: <file_image> <file_image> <file_image> <file_image> <file_image> <file_image> <file_image> <file_image><file_image>    Omari: Thanks bae    Tori: np    Omari: What about those we took at mall    Tori: Here you go <file_image> <file_image> <file_image> <file_image> <file_image>    Omari: <3    Tori: Were there some more?    Omari: Yes Ethan have them    Tori: I will ask him to send me
Adam: <file_other>    Audrey: What an annoying buffoon    Adam: "I cannot in good conscience support the terms proposed for our deal with the EU."    Audrey: What did he expect?    Adam: I think the main problem of these people 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
response = openai.Completion.create(
        model = "text-davinci-003",
        prompt = "Summarize this: " + df.iloc[0,0],
        max_tokens = 32,
        temperature=0)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nAmanda offered to bake cookies for Jerry and he accepted. She will bring them to him tomorrow."
    }
  ],
  "created": 1681864568,
  "id": "cmpl-76q5o9HGPDB6fjooQGdCtQMV7Ai5F",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 22,
    "prompt_tokens": 31,
    "total_tokens": 53
  }
}


In [12]:
t = response.choices[0].text
t

'\n\nAmanda offered to bake cookies for Jerry and he accepted. She will bring them to him tomorrow.'

In [13]:
cand = t.replace("\n\n", "")
cand

'Amanda offered to bake cookies for Jerry and he accepted. She will bring them to him tomorrow.'

In [14]:
ref = df.iloc[0, 1]

In [15]:
ref

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [19]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL', 'rouge2'], use_stemmer=True)

In [20]:
scores = scorer.score(cand, ref)

In [21]:
scores

{'rouge1': Score(precision=0.8888888888888888, recall=0.47058823529411764, fmeasure=0.6153846153846153),
 'rougeL': Score(precision=0.7777777777777778, recall=0.4117647058823529, fmeasure=0.5384615384615384),
 'rouge2': Score(precision=0.25, recall=0.125, fmeasure=0.16666666666666666)}